In [21]:
import tkinter as tk
from tkinter import messagebox
import joblib
import numpy as np
import pandas as pd
import shap
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg

# 加载模型
model = joblib.load('XGBoost.pkl')

# GUI应用程序类
class HeartDiseasePredictorApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Heart Disease Predictor")
        
        # 特征名称和分类选项的字典
        self.entries = {}
        self.feature_names = ['age', 'sex', 'cp', 'trestbps', 'chol', 'fbs', 'restecg', 'thalach', 'exang', 'oldpeak', 'slope', 'ca', 'thal']

        # 创建输入框和下拉菜单
        input_frame = tk.Frame(root)
        input_frame.grid(row=0, column=0, padx=10, pady=10)

        # age: 数值输入
        self.create_label_and_entry(input_frame, 'age', 'Age:', 0)

        # sex: 分类选择
        self.create_label_and_optionmenu(input_frame, 'sex', 'Sex (0=Female, 1=Male):', {0: 'Female (0)', 1: 'Male (1)'}, 1)

        # cp: 分类选择
        cp_options = {
            1: 'Typical angina (1)',
            2: 'Atypical angina (2)',
            3: 'Non-anginal pain (3)',
            4: 'Asymptomatic (4)'
        }
        self.create_label_and_optionmenu(input_frame, 'cp', 'Chest pain type:', cp_options, 2)

        # trestbps: 数值输入
        self.create_label_and_entry(input_frame, 'trestbps', 'Resting blood pressure (trestbps):', 3)

        # chol: 数值输入
        self.create_label_and_entry(input_frame, 'chol', 'Serum cholestoral in mg/dl (chol):', 4)

        # fbs: 分类选择
        self.create_label_and_optionmenu(input_frame, 'fbs', 'Fasting blood sugar > 120 mg/dl (fbs):', {0: 'False (0)', 1: 'True (1)'}, 5)

        # restecg: 分类选择
        restecg_options = {
            0: 'Normal (0)',
            1: 'ST-T wave abnormality (1)',
            2: 'Left ventricular hypertrophy (2)'
        }
        self.create_label_and_optionmenu(input_frame, 'restecg', 'Resting electrocardiographic results:', restecg_options, 6)

        # thalach: 数值输入
        self.create_label_and_entry(input_frame, 'thalach', 'Maximum heart rate achieved (thalach):', 7)

        # exang: 分类选择
        self.create_label_and_optionmenu(input_frame, 'exang', 'Exercise induced angina (exang):', {0: 'No (0)', 1: 'Yes (1)'}, 8)

        # oldpeak: 数值输入
        self.create_label_and_entry(input_frame, 'oldpeak', 'ST depression induced by exercise relative to rest (oldpeak):', 9)

        # slope: 分类选择
        slope_options = {
            1: 'Upsloping (1)',
            2: 'Flat (2)',
            3: 'Downsloping (3)'
        }
        self.create_label_and_optionmenu(input_frame, 'slope', 'Slope of the peak exercise ST segment (slope):', slope_options, 10)

        # ca: 数值输入
        self.create_label_and_entry(input_frame, 'ca', 'Number of major vessels colored by fluoroscopy (ca):', 11)

        # thal: 分类选择
        thal_options = {
            3: 'Normal (3)',
            6: 'Fixed defect (6)',
            7: 'Reversible defect (7)'
        }
        self.create_label_and_optionmenu(input_frame, 'thal', 'Thal (thal):', thal_options, 12)

        # 创建按钮
        predict_button = tk.Button(root, text="Predict", command=self.predict)
        predict_button.grid(row=1, column=0, pady=10)

        # 用于显示结果的文本框
        self.result_text = tk.Text(root, height=10, width=50)
        self.result_text.grid(row=2, column=0, padx=10, pady=10)

        # 用于显示SHAP图的按钮
        self.shap_button = tk.Button(root, text="Show SHAP Force Plot", command=self.show_shap_plot, state=tk.DISABLED)
        self.shap_button.grid(row=3, column=0, pady=10)

    def create_label_and_entry(self, frame, feature_name, label_text, row):
        """Create a label and entry widget."""
        label = tk.Label(frame, text=label_text)
        label.grid(row=row, column=0, padx=10, pady=5)
        entry = tk.Entry(frame)
        entry.grid(row=row, column=1, padx=10, pady=5)
        self.entries[feature_name] = entry

    def create_label_and_optionmenu(self, frame, feature_name, label_text, options, row):
        """Create a label and option menu widget."""
        label = tk.Label(frame, text=label_text)
        label.grid(row=row, column=0, padx=10, pady=5)
        var = tk.StringVar(frame)
        var.set(list(options.keys())[0])  # 设置默认值
        optionmenu = tk.OptionMenu(frame, var, *options.keys())
        optionmenu.grid(row=row, column=1, padx=10, pady=5)
        self.entries[feature_name] = var

    def predict(self):
        try:
            # 获取用户输入的特征值
            feature_values = []
            for feature in self.feature_names:
                value = self.entries[feature].get()
                feature_values.append(float(value))
            features = np.array([feature_values])

            # 预测类别和概率
            predicted_class = model.predict(features)[0]
            predicted_proba = model.predict_proba(features)[0]

            # 显示预测结果
            result = f"Predicted Class: {predicted_class}\n"
            result += f"Prediction Probabilities: {predicted_proba}\n"
            self.result_text.delete(1.0, tk.END)
            self.result_text.insert(tk.END, result)

            # 根据预测结果生成建议
            advice = self.generate_advice(predicted_class, predicted_proba)
            self.result_text.insert(tk.END, advice)

            # 启用SHAP按钮
            self.shap_button.config(state=tk.NORMAL)

            # 计算SHAP值并保存力图为图像文件
            self.save_shap_force_plot(feature_values)
        except Exception as e:
            messagebox.showerror("Error", f"An error occurred: {e}")

    def generate_advice(self, predicted_class, predicted_proba):
        """根据预测结果生成建议."""
        probability = predicted_proba[predicted_class] * 100

        if predicted_class == 1:
            advice = (
                f"\n根据我们的模型预测，您的心脏疾病的风险很高。"
                f"模型预测您患有心脏疾病的可能性为{probability:.1f}%。"
                "虽然这只是一个概率估计，但这表明您可能存在较高的心脏疾病风险。"
                "我建议您尽快联系心脏专科医生进行进一步的检查和评估，"
                "以确保得到准确的诊断和必要的治疗措施。\n"
            )
        else:
            advice = (
                f"\n根据我们的模型预测，您的心脏疾病风险较低。"
                f"模型预测您未患有心脏疾病的可能性为{probability:.1f}%。"
                "尽管如此，保持健康的生活方式仍然非常重要。"
                "建议您定期进行体检，以监测心脏健康，"
                "并在有任何不适症状时及时就医。\n"
            )

        return advice

    def save_shap_force_plot(self, feature_values):
        # 准备特征数据
        features_df = pd.DataFrame([feature_values], columns=self.feature_names)

        # 创建SHAP解释器
        explainer = shap.TreeExplainer(model)

        # 计算SHAP值
        shap_values = explainer.shap_values(features_df)

        # 绘制SHAP力图
        plt.figure()
        shap.force_plot(explainer.expected_value, shap_values[0], features_df.iloc[0], matplotlib=True, show=False)

        # 保存SHAP力图为PNG
        self.force_plot_path = "shap_force_plot.png"
        plt.savefig(self.force_plot_path, bbox_inches='tight', dpi=300)
        plt.close()

    def show_shap_plot(self):
        # 在新的窗口中展示SHAP力图
        force_window = tk.Toplevel(self.root)
        force_window.title("SHAP力图")

        # 读取并展示保存的SHAP力图
        fig, ax = plt.subplots(figsize=(10, 7))
        img = plt.imread(self.force_plot_path)
        ax.imshow(img)
        ax.axis('off')  # 隐藏坐标轴
        canvas = FigureCanvasTkAgg(fig, master=force_window)
        canvas.draw()
        canvas.get_tk_widget().pack()

# 创建应用程序窗口
root = tk.Tk()
app = HeartDiseasePredictorApp(root)

# 运行应用程序
root.mainloop()